In [7]:
using Knet: Knet, dir, accuracy, progress, sgd, load, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout
using Statistics
using CUDA
using Random
using LinearAlgebra
using Knet

┌ Warning: Your Tesla K20m GPU does not meet the minimal required compute capability (3.5.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA /kuacc/users/hcoban15/.julia/packages/CUDA/YeS8q/src/state.jl:251


In [11]:
struct Chain
    layers;
    Chain(layers...) = new(layers);
end

(c::Chain)(x) = (for l in c.layers; x = l(x); end; x);
(c::Chain)(x,y) = loss(c(x),y);
(c::Chain)(x::Tuple) = loss(c(x[1]),x[2]);
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d);

struct Dense
    w;
    b;
    f;
end

Dense(i::Int,o::Int,f=relu) = Dense(param(o,i),param0(o),f);
(l::Dense)(x) = (l.w * x .+ l.b);




In [12]:
function loss(Q_value,cost)
      return mean((Q_value-cost).^2)/2;
end
function sgdupdate!(func, x,y; lr=0.1)
    fval = @diff func(x,y);
    for param in params(fval)
        ∇param = grad(fval, param);
        param .-= lr * ∇param;
    end
    return value(fval);
end


sgdupdate! (generic function with 1 method)